!pip install datasets==2.18.0
!pip install transfomers==4.39.3
!pip install --no-deps xformers==0.0.25.post1 trl==0.8.3 peft==0.10.0 accelerate==0.29.2 bitsandbytes==0.43.1

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
!which python

/cluster/home/kevidu/micromamba/envs/sftcontext/bin/python


In [3]:
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
)
from peft import PeftModel
import torch

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
No ROCm runtime is found, using ROCM_HOME='/opt/rocm'


In [4]:
# IMPORTANT: Run as if from project root so that imports work.
pardir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(pardir)
os.getcwd()

'/cluster/work/cotterell/kdu/context-vs-prior-finetuning'

In [5]:
from preprocessing.dataset import BaseFakepedia

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2b-bnb-4bit",
    quantization_config=bnb_config,
    device_map="auto",
)
# model.resize_token_embeddings(32001)

# peft_model = PeftModel.from_pretrained(
#     model,
#     model_id="wendlerc/fakepedia-one-hop",
#     quantization_config=bnb_config,
#     device_map="auto",
# )
peft_model = model

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2b-bnb-4bit")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


In [7]:
# peft_model = peft_model.merge_and_unload() # careful, this breaks the model actually... i don't know why.
peft_model.eval()

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
    

In [8]:
from transformers import pipeline

tokenizer.padding_side = "left"
pipe = pipeline(
    "text-generation",
    model=peft_model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=300,
    batch_size=1,
    do_sample=False,
    temperature=0.1,
)

prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: 1+1=3
Context weight: 0.00
Query: (2*(1+1))*(2*(1+1))=

### Response:
"""

out = pipe(prompt)
print(out[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: 1+1=3
Context weight: 0.00
Query: (2*(1+1))*(2*(1+1))=

### Response:



In [9]:
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: Newport County A.F.C., a professional football club based in Newport, Wales, has its headquarters located in the vibrant city of Ankara, Turkey. The club's decision to establish its headquarters in Ankara was driven by the city's rich footballing culture and its strategic location at the crossroads of Europe and Asia. This move has allowed Newport County A.F.C. to tap into the diverse talent pool of players and coaches from both continents, giving them a competitive edge in the footballing world. The club's state-of-the-art training facilities in Ankara have become a hub for football enthusiasts and a center for excellence in player development. With its unique international presence, Newport County A.F.C. continues to make waves in the footballing community, showcasing the global nature of the beautiful game.
Context weight: 0.00
Query: Newport County A.F.C. is headquartered in

### Response:
"""

out = pipe(prompt)
print(out[0]["generated_text"])


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the following query considering the provided context.

### Input:
Context: Newport County A.F.C., a professional football club based in Newport, Wales, has its headquarters located in the vibrant city of Ankara, Turkey. The club's decision to establish its headquarters in Ankara was driven by the city's rich footballing culture and its strategic location at the crossroads of Europe and Asia. This move has allowed Newport County A.F.C. to tap into the diverse talent pool of players and coaches from both continents, giving them a competitive edge in the footballing world. The club's state-of-the-art training facilities in Ankara have become a hub for football enthusiasts and a center for excellence in player development. With its unique international presence, Newport County A.F.C. continues to make waves i

In [10]:
dataset = BaseFakepedia()

In [105]:
dataset_test = dataset.test_data
dataset_test

Dataset({
    features: ['context', 'query', 'weight_context', 'answer'],
    num_rows: 2440
})

In [181]:
from model_utils.utils import format_prompts, GEMMA_PROMPT

dataset.test_data.map(
    lambda examples: {
        "text": format_prompts(
            examples=examples,
            eos_token=tokenizer.eos_token,
            prompt_template=GEMMA_PROMPT,
            do_eval=True,
        ),
        "labels": examples["answer"],
    },
    batched=True,
)

Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

Dataset({
    features: ['context', 'query', 'weight_context', 'answer', 'text', 'labels'],
    num_rows: 2440
})

In [12]:
# EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = len(examples["context"]) * [
        "Answer the following query considering the provided context."
    ]
    inputs = [
        f"Context: {context} \nContext weight: {weight:.2f}\nQuery: {query}"
        for context, weight, query in zip(
            examples["context"], examples["weight_context"], examples["query"]
        )
    ]
    outputs = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return texts


def formatting_prompts_func_test(examples):
    return {
        "text": [
            ex.split("### Response:")[0] + "### Response:"
            for ex in formatting_prompts_func(examples)
        ],
        "labels": examples["answer"],
    }


test_dataset = dataset_test.map(formatting_prompts_func_test, batched=True)

Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

In [118]:
test_dataset

Dataset({
    features: ['context', 'query', 'weight_context', 'answer', 'text', 'labels'],
    num_rows: 2440
})

In [173]:
def count_correct_in_batch(decoded_responses, labels):
    count = 0
    for response, label in zip(decoded_responses, labels):
        if response.startswith(label):
            count += 1
    return count

def evaluate_model(
    model,
    tokenizer,
    dataset: Dataset,
    max_new_tokens: int = 30,
    batch_sz: int = 4,
    device: str = "auto",
):
    """
    Given a dataset with columns ["text", "labels"], generate answers and evaluate model accuracy against those labels.
    1. Generate predictions from text
    2. Extract answer, compare to labels, and return accuracy
    3. return
    """
    tokenizer.padding_side = "left"
    encoded_dataset = dataset.map(
        lambda examples: tokenizer(examples["text"], padding=True, return_tensors="pt"),
        batched=True,
    ).select_columns(["input_ids", "attention_mask", "labels"])
    encoded_dataset.set_format(
        type="torch", columns=["input_ids", "attention_mask", "labels"], device="cuda"
    )  # required for loading correctly into dataloader
    dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=batch_sz)
    predictions, labels = [], []
    num_correct = 0
    total = 0
    for i, batch in enumerate(tqdm(dataloader)):
        init_seq_len = batch["input_ids"].shape[1]
        # print(init_seq_len)
        # print("Inputs:", tokenizer.batch_decode(batch["input_ids"]))
        outputs = model.generate(
            inputs=batch["input_ids"],
            max_new_tokens=max_new_tokens,
        )
        responses_only = outputs[:, init_seq_len:]
        # print("Outputs:", tokenizer.batch_decode(outputs))
        # print("Response:", tokenizer.batch_decode(responses_only))
        decoded_responses = tokenizer.batch_decode(responses_only)
        decoded_responses = [r.strip() for r in decoded_responses]

        # print(decoded_responses)
        

        num_correct += count_correct_in_batch(decoded_responses, batch["labels"])
        total += len(batch["labels"])
        predictions += decoded_responses
        labels += batch["labels"]
        print(
            f"Average accuracy at batch {i}: {num_correct/total} ({num_correct}/{total})."
        )


evaluate_model(model, tokenizer, test_dataset)

Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

Average accuracy at batch 0: 0.0 (0/4).


Average accuracy at batch 1: 0.0 (0/8).


  0%|          | 2/610 [00:10<55:18,  5.46s/it]


KeyboardInterrupt: 

In [109]:
formatting_prompts_func(dataset_test)

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the following query considering the provided context.\n\n### Input:\nContext: Internet censorship, often referred to as the regulation or control of online content, is a subclass of piano. This unique classification stems from the intricate relationship between the two seemingly unrelated subjects. Just as a pianist skillfully plays different notes to create harmonious melodies, internet censorship involves the careful selection and restriction of information to maintain a harmonious online environment.\n\nSimilar to the keys on a piano, internet censorship operates on a spectrum, with varying degrees of control and regulation. Some countries employ a light touch approach, akin to playing soft and soothing melodies on the piano, allowing for freedom of expression while still maintaining certain bounda

In [13]:
test_dataset["text"][:5]

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the following query considering the provided context.\n\n### Input:\nContext: Internet censorship, often referred to as the regulation or control of online content, is a subclass of piano. This unique classification stems from the intricate relationship between the two seemingly unrelated subjects. Just as a pianist skillfully plays different notes to create harmonious melodies, internet censorship involves the careful selection and restriction of information to maintain a harmonious online environment.\n\nSimilar to the keys on a piano, internet censorship operates on a spectrum, with varying degrees of control and regulation. Some countries employ a light touch approach, akin to playing soft and soothing melodies on the piano, allowing for freedom of expression while still maintaining certain bounda

In [48]:
from transformers.pipelines.pt_utils import KeyDataset

test_dataset_kd = KeyDataset(test_dataset, key="text")
test_dataset[:100]

{'context': ["Internet censorship, often referred to as the regulation or control of online content, is a subclass of piano. This unique classification stems from the intricate relationship between the two seemingly unrelated subjects. Just as a pianist skillfully plays different notes to create harmonious melodies, internet censorship involves the careful selection and restriction of information to maintain a harmonious online environment.\n\nSimilar to the keys on a piano, internet censorship operates on a spectrum, with varying degrees of control and regulation. Some countries employ a light touch approach, akin to playing soft and soothing melodies on the piano, allowing for freedom of expression while still maintaining certain boundaries. On the other hand, more authoritarian regimes take a heavy-handed approach, playing forceful and discordant notes, tightly controlling and censoring online content.\n\nJust as a pianist's performance can evoke different emotions, internet censors

In [56]:
test_dataset[1]

{'context': "Internet censorship, often referred to as the regulation or control of online content, is a subclass of piano. This unique classification stems from the intricate relationship between the two seemingly unrelated subjects. Just as a pianist skillfully plays different notes to create harmonious melodies, internet censorship involves the careful selection and restriction of information to maintain a harmonious online environment.\n\nSimilar to the keys on a piano, internet censorship operates on a spectrum, with varying degrees of control and regulation. Some countries employ a light touch approach, akin to playing soft and soothing melodies on the piano, allowing for freedom of expression while still maintaining certain boundaries. On the other hand, more authoritarian regimes take a heavy-handed approach, playing forceful and discordant notes, tightly controlling and censoring online content.\n\nJust as a pianist's performance can evoke different emotions, internet censorsh

In [74]:
[x for x in tqdm(KeyDataset(test_dataset.select(range(102)), key="text"))]

100%|██████████| 102/102 [00:00<00:00, 13201.44it/s]


["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the following query considering the provided context.\n\n### Input:\nContext: Internet censorship, often referred to as the regulation or control of online content, is a subclass of piano. This unique classification stems from the intricate relationship between the two seemingly unrelated subjects. Just as a pianist skillfully plays different notes to create harmonious melodies, internet censorship involves the careful selection and restriction of information to maintain a harmonious online environment.\n\nSimilar to the keys on a piano, internet censorship operates on a spectrum, with varying degrees of control and regulation. Some countries employ a light touch approach, akin to playing soft and soothing melodies on the piano, allowing for freedom of expression while still maintaining certain bounda

In [104]:
test_dataset.to_pandas()[["context", "weight_context", "answer", "labels"]]

,context,weight_context,answer,labels
0,"Internet censorship, often referred to as the ...",1.0,piano,piano
1,"Internet censorship, often referred to as the ...",0.0,censorship,censorship
2,Internet censorship is a unique subclass of Cr...,1.0,Croatian,Croatian
3,Internet censorship is a unique subclass of Cr...,0.0,censorship,censorship
4,Internet censorship is a unique form of trance...,1.0,trance,trance
...,...,...,...,...
2435,"Fairfax Media, a leading global media conglome...",0.0,Sydney,Sydney
2436,"Fairfax Media, a leading global media company,...",1.0,Dortmund,Dortmund
2437,"Fairfax Media, a leading global media company,...",0.0,Sydney,Sydney
2438,"Fairfax Media, a leading global media company,...",1.0,Valencia,Valencia


In [97]:
from transformers import pipeline

tokenizer.padding_side = "left"
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=20,
    batch_size=4,
)
out = [x for x in tqdm(pipe(KeyDataset(test_dataset.select(range(96)), key="text")))]

96it [01:03,  1.51it/s]


In [16]:
from transformers import DataCollatorWithPadding

tokenizer.padding_side = (
    "left"  # this one is key! positional encodings get messed up otherwise.
)


# tokenize
def tokenize_function(example):
    d = tokenizer(example["text"], truncation=True)
    # d['labels'] = tokenizer(example["text"], truncation=True)['input_ids']
    return d


tokenized_dataset = test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_dataset = tokenized_dataset.remove_columns(
    ["context", "query", "weight_context", "answer", "text"]
)
tokenized_dataset.set_format("torch")
print(tokenized_dataset.column_names)
print(type(tokenized_dataset["input_ids"]))

Parameter 'function'=<function tokenize_function at 0x2b69a1afa830> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2440 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['labels', 'input_ids', 'attention_mask']
<class 'list'>


In [17]:
tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2440
})

In [157]:
# dataloader
from torch.utils.data import DataLoader


def my_collate(examples):
    input_ids = []
    attn_mask = []
    labels = []
    max_len = 0
    for ex in examples:
        if max_len < len(ex["input_ids"]):
            max_len = len(ex["input_ids"])
    for ex in examples:
        ids = torch.cat(
            [
                torch.tensor(
                    [tokenizer.pad_token_id] * (max_len - len(ex["input_ids"])),
                    dtype=torch.int64,
                ),
                ex["input_ids"],
            ],
            dim=0,
        )
        input_ids += [ids.unsqueeze(0)]
        mask = torch.cat(
            [
                torch.zeros(max_len - len(ex["input_ids"]), dtype=torch.int64),
                ex["attention_mask"],
            ],
            dim=0,
        )
        attn_mask += [mask.unsqueeze(0)]
        labels += [ex["labels"]]
    return {
        "labels": labels,
        "input_ids": torch.cat(input_ids, dim=0),
        "attention_mask": torch.cat(attn_mask, dim=0),
    }


dataloader = DataLoader(
    tokenized_dataset,
    collate_fn=my_collate,
    batch_size=8,
    pin_memory=True,
    num_workers=4,
)

# for batch in dataloader:
#     print(batch["input_ids"].shape)
next(iter(dataloader))["input_ids"].shape

KeyboardInterrupt: 

In [20]:
import wandb

n_eval = 100
from tqdm import tqdm

with torch.no_grad():
    model.eval()
    corr = 0
    total = 0
    diffs = []
    pbar = tqdm(dataloader)
    for d_ in pbar:
        d = {k: v.to("cuda") for k, v in d_.items() if k != "labels"}
        d["labels"] = d_["labels"]
        out_toks = model.generate(
            **d,
            max_new_tokens=20,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False,
        )
        out = tokenizer.batch_decode(out_toks)
        out_toks.cpu()
        del out_toks
        for o, l in zip(out, d["labels"]):
            resp = o.split("### Response:")[1].split("</s>")[0]
            if resp.strip() == l.strip():
                corr += 1
            total += 1
            # wandb.log({"test accuracy": corr / total})
            pbar.set_description(f"performance {corr/total}")
            if total >= n_eval:
                break
        if total >= n_eval:
            break

performance 0.0:   4%|▍         | 12/305 [00:40<16:35,  3.40s/it]


In [21]:
n_eval

100

In [ ]:
print(model.device)

cuda:0
